In [1]:
import pika
import uuid
import time
from utils import NumpyEncoder
import json

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
exchangeName = 'direct_pings'

In [3]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_pings	direct


In [4]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-RhuLmneWWdCahw5RvWJBAQ	queue	amq.gen-RhuLmneWWdCahw5RvWJBAQ	[]
	exchange	amq.gen-_fNZavjKI_BhNaV4d-fOBw	queue	amq.gen-_fNZavjKI_BhNaV4d-fOBw	[]
direct_pings	exchange	amq.gen-RhuLmneWWdCahw5RvWJBAQ	queue	ping_queue	[]
direct_pings	exchange	amq.gen-_fNZavjKI_BhNaV4d-fOBw	queue	ping_queue	[]


In [5]:
!sudo rabbitmqctl list_channels

Listing channels ...
<rabbit@ip-172-31-14-37.3.5652.9>	myguest	1	0
<rabbit@ip-172-31-14-37.3.5897.9>	myguest	1	0


Below we will create a connection, channel, and queue

**queue_declare(queue, passive=False, durable=False, exclusive=False, auto_delete=False, arguments=None, callback=None)**<br>
<br>
Declare queue, create if needed. This method creates or checks a queue. When creating a new queue the client can specify various properties that control the durability of the queue and its contents, and the level of sharing for the queue.

**basic_consume(queue, on_message_callback, auto_ack=False, exclusive=False, consumer_tag=None, arguments=None, callback=None)**

Sends the AMQP 0-9-1 command Basic.Consume to the broker and binds messages for the consumer_tag to the consumer callback. If you do not pass in a consumer_tag, one will be automatically generated for you. Returns the consumer tag.

**queue (str)** – The queue to consume from. Use the empty string to specify the most recent server-named queue for this channel

**on_message_callback**: (callable) –
The function to call when consuming with the **signature on_message_callback(channel, method, properties, body)**, where

* **channel**: pika.Channel 

* **method**: pika.spec.Basic.Deliver 

* **properties**: pika.spec.BasicProperties 

* **body**: bytes


### Message properties

The AMQP 0-9-1 protocol predefines a set of 14 properties that go with a message. Most of the properties are rarely used, with the exception of the following:

* delivery_mode: Marks a message as persistent (with a value of 2) or transient (any other value). 
* content_type: Used to describe the mime-type of the encoding. For example for the often used JSON encoding it is a good practice to set this property to: application/json.
* reply_to: Commonly used to name a callback queue.
* correlation_id: Useful to correlate RPC responses with requests.

**See https://www.rabbitmq.com/tutorials/tutorial-six-python.html for details**

In [6]:
class TalkToWorkersClient(object):

    def __init__(self):
        #self.connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

        self.connection = pika.BlockingConnection(parameters)
        self.channel = self.connection.channel()
        self.channel.exchange_declare(exchange=exchangeName, exchange_type='direct')
        
        self.responses=[]
        
        #Create a new queue with a random name. We will use this queue to receive messages (consume)
        myqueue = self.channel.queue_declare('', exclusive=True)
        print("myqueue:", myqueue)
        
        self.callback_queue = myqueue.method.queue #The queue we just created

        print("myqueue.method:", myqueue.method)
        
        # Let us start waiting for messages (The results of our call). 
        #When we recevie a message, we will the on_response call back function
        self.channel.basic_consume(
            queue=self.callback_queue, #the queue we created above
            on_message_callback=self.on_response, #The callback function to run when we receive a message from the workers
            auto_ack=True)
            
    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id: #Check if our correlation id is the same as we get in the message
            self.response = body
            self.responses.append(body)
            print(self.response)
            
            
            

    def send_task2worker(self, worker_routing,n):
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id)
        
        json_dump = json.dumps({'task': 'Fib','num':n}, cls=NumpyEncoder) 
        # Now we are ready to send our message to the workers
        self.channel.basic_publish(
            exchange=exchangeName,
            routing_key=worker_routing,
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
                delivery_mode =1,
                content_type ="text/plain"
            ),
            body=json_dump)
        
        #Waiting for the results
        while self.response is None:
            self.connection.process_data_events()
        return self.response
        
    def ping_workers(self):
        self.responses=[]
        self.response = None
        self.corr_id = str(uuid.uuid4()) # This is where we create our unique correlation id
        print('corr_id:', self.corr_id)
        json_dump = json.dumps({'task': 'Ping!'}, cls=NumpyEncoder) 
        # Now we are ready to send our "Ping" message to the workers
        self.channel.basic_publish(
            exchange=exchangeName,
            routing_key='ping_queue', #sending messages on the "ping_queue" routing.
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
                delivery_mode =1,
                content_type ="text/plain"
            ),
            body=json_dump)
        
        
        t_end = time.time() + 5
        while time.time() < t_end:
        #while self.response is None:
        #Waiting for the results
           self.connection.process_data_events()
        return self.response
    

In [7]:
#This will create one channel and queue. All communications will be done on that queue using different correlation ids. 
talkToWorkersClient_instance = TalkToWorkersClient()

myqueue: <METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-5hNzVPzjk0HHuj9TCfVvHw'])>"])>
myqueue.method: <Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=amq.gen-5hNzVPzjk0HHuj9TCfVvHw'])>


In [8]:
response = talkToWorkersClient_instance.ping_workers()
print(response)

corr_id: 9d6accb0-057c-49ea-a0e9-90c0ec2ac554
b'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}'
b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}'
b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}'


In [9]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-5hNzVPzjk0HHuj9TCfVvHw	queue	amq.gen-5hNzVPzjk0HHuj9TCfVvHw	[]
	exchange	amq.gen-RhuLmneWWdCahw5RvWJBAQ	queue	amq.gen-RhuLmneWWdCahw5RvWJBAQ	[]
	exchange	amq.gen-_fNZavjKI_BhNaV4d-fOBw	queue	amq.gen-_fNZavjKI_BhNaV4d-fOBw	[]
direct_pings	exchange	amq.gen-RhuLmneWWdCahw5RvWJBAQ	queue	ping_queue	[]
direct_pings	exchange	amq.gen-RhuLmneWWdCahw5RvWJBAQ	queue	routing_172.31.14.37_11770	[]
direct_pings	exchange	amq.gen-_fNZavjKI_BhNaV4d-fOBw	queue	ping_queue	[]
direct_pings	exchange	amq.gen-_fNZavjKI_BhNaV4d-fOBw	queue	routing_172.31.14.37_12986	[]


In [10]:
talkToWorkersClient_instance.responses

[b'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}',
 b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}']

In [11]:
response = talkToWorkersClient_instance.ping_workers()
print(response)

corr_id: d9638c91-5348-4108-8810-0d6b44a033ce
b'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}'
b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}'
b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}'


In [12]:
talkToWorkersClient_instance.responses

[b'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}',
 b'{"task": "Ping!", "result": "routing_172.31.14.37_12986"}']

In [13]:
!sudo rabbitmqctl list_channels

Listing channels ...
<rabbit@ip-172-31-14-37.3.5652.9>	myguest	1	0
<rabbit@ip-172-31-14-37.3.5897.9>	myguest	1	0
<rabbit@ip-172-31-14-37.3.6839.9>	myguest	1	0


In [14]:
talkToWorkersClient_instance.responses[0].decode('ascii')

'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}'

In [15]:
worker_routing=json.loads(talkToWorkersClient_instance.responses[0])['result']

In [16]:
talkToWorkersClient_instance.send_task2worker(worker_routing,14)

corr_id: d57cdcaf-2201-42be-8829-f8b811f5e50a
b'{"task": "Fib", "result": "377"}'


b'{"task": "Fib", "result": "377"}'

In [17]:
talkToWorkersClient_instance.responses[0]

b'{"task": "Ping!", "result": "routing_172.31.14.37_11770"}'

In [18]:
worker_routing=json.loads(talkToWorkersClient_instance.responses[1])['result']
print(worker_routing)

routing_172.31.14.37_12986


In [19]:
talkToWorkersClient_instance.send_task2worker(worker_routing,12)

corr_id: 5facc5bc-ae7b-422b-bbcf-459472a2466e
b'{"task": "Fib", "result": "144"}'


b'{"task": "Fib", "result": "144"}'